In [29]:
import re

In [30]:
class CPU:
    name:str    # c0~c#
    usr:str   # %
    sys:str   # %
    idle:str  # %

class TOP:
    time        :str    # sec
    mem_total   :str    # MB
    mem_free    :str    # MB
    mem_used    :str    # MB
    mem_buff   :str    # MB
    cpus        :list   # # of CPU


In [31]:
def get_sec(time_str):
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

In [32]:
def parseCPU(cpu:str):
    raw = CPU()
    raw.name = 'c' + re.findall(r'%Cpu([\d])\s', cpu)[0]
    raw.usr = re.findall(r'([\d.]+)\sus,', cpu)[0]
    raw.sys = re.findall(r'([\d.]+)\ssy,', cpu)[0]
    raw.idle = re.findall(r'([\d.]+)\sid,', cpu)[0]
    return raw

In [33]:
def parseTop(top:list):
    raw = TOP()
    cpus = []
    try:
        for line in top:
            if not line : continue
            print(line)
            if 'top' in line:
                time = re.findall(r'top - ([\d\:]+)\s', line)[0]
                raw.time = get_sec(time)
# MiB Mem :  63992.4 total,  61707.1 free,   1268.8 used,   1016.6 buff/
            elif 'MiB' in line:
                raw.mem_total = re.findall(r'\s([\d.]+)\stotal,', line)[0]
                raw.mem_free = re.findall(r'\s([\d.]+)\sfree,', line)[0]
                raw.mem_used = re.findall(r'\s([\d.]+)\sused,', line)[0]
                raw.mem_buff = re.findall(r'\s([\d.]+)\sbuff\/', line)[0]
            else:
                cpus.append(parseCPU(line))
            # raw.cpus.append(cpu)
        
        raw.cpus = cpus
        return raw
    except:
        # print(top)
        print('except: ', raw.__dict__)

In [34]:
def parseTrace(filename):
    infile = open(filename, 'r')

    lines=[]
    result=[]
    for line in infile.readlines():
        if not line : continue
        # print(idx, line)
        if('top' in line):
            if(len(lines) > 1):
                result.append(parseTop(lines))
                lines.clear()
        lines.append(line)
        
    infile.close()
    outfile = open(filename + '.parse', 'w')

    # Header
    outfile.write('time,mem_total,mem_free,mem_used,mem_buff,')
    for cpu in result[0].cpus:
        outfile.write(cpu.name + '_usr,')
        outfile.write(cpu.name + '_sys,')
        outfile.write(cpu.name + '_idle,')
    outfile.write('\n')

    # Data
    startTime = result[0].time
    for stamp in result:
        outfile.write(str(stamp.time - startTime) + ',')
        outfile.write(stamp.mem_total + ',')
        outfile.write(stamp.mem_free + ',')
        outfile.write(stamp.mem_used + ',')
        outfile.write(stamp.mem_buff + ',')
        for cpu in stamp.cpus:
            outfile.write(cpu.usr + ',')
            outfile.write(cpu.sys + ',')
            outfile.write(cpu.idle + ',')
        outfile.write('\n')
    # for cpu in result[0].cpus:
    #     outfile.write(f'{cpu.name}_usr,{cpu.name}_sys,{cpu.name}_idle\n')

    # for item in result:
    #     # outfile.write(item.)
    #     outfile.write(','.join(item.__dict__.values()) + '\n')
    
    outfile.close()

    return outfile.name

In [35]:
filename = './raw/top.txt'
parsedFile = parseTrace(filename)

top - 00:54:32 up 17 min,  6 users,  load average: 0.39, 5.03, 3.90

%Cpu0  : 12.5 us, 43.8 sy,  0.0 ni, 43.8 id,  0.0 wa,  0.0 hi,  0.0 si,

%Cpu1  :  5.9 us, 11.8 sy,  0.0 ni, 82.4 id,  0.0 wa,  0.0 hi,  0.0 si,

%Cpu2  :  0.0 us, 11.8 sy,  0.0 ni, 88.2 id,  0.0 wa,  0.0 hi,  0.0 si,

%Cpu3  :  0.0 us,  5.6 sy,  0.0 ni, 94.4 id,  0.0 wa,  0.0 hi,  0.0 si,

%Cpu4  :  5.9 us,  5.9 sy,  0.0 ni, 88.2 id,  0.0 wa,  0.0 hi,  0.0 si,

%Cpu5  :  5.6 us, 11.1 sy,  0.0 ni, 83.3 id,  0.0 wa,  0.0 hi,  0.0 si,

%Cpu6  :  6.2 us, 18.8 sy,  0.0 ni, 75.0 id,  0.0 wa,  0.0 hi,  0.0 si,

%Cpu7  :  0.0 us,  0.0 sy,  0.0 ni,100.0 id,  0.0 wa,  0.0 hi,  0.0 si,

MiB Mem :  63992.4 total,  61707.1 free,   1268.8 used,   1016.6 buff/c

top - 00:54:33 up 17 min,  5 users,  load average: 0.39, 5.03, 3.90

%Cpu0  :  0.0 us, 58.8 sy,  0.0 ni, 41.2 id,  0.0 wa,  0.0 hi,  0.0 si,

%Cpu1  :  0.0 us,  0.0 sy,  0.0 ni,100.0 id,  0.0 wa,  0.0 hi,  0.0 si,

%Cpu2  :  0.0 us,  6.2 sy,  0.0 ni, 93.8 id,  0.0 wa,  0.0 